In [1]:
import numpy as np
import cv2
import math as m
from matplotlib import pyplot as plt

In [17]:
im_gray = cv2.imread('fingerprint.jpg', cv2.IMREAD_GRAYSCALE)
(thresh, im_bw) = cv2.threshold(im_gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
thresh = 127
im_bw = cv2.threshold(im_gray, thresh, 255, cv2.THRESH_BINARY)[1]
cv2.imwrite('fingerprint_bw.png', im_bw)

True

In [3]:
test = np.array([[0,0,0,0,0,0,255,0,0],
                 [0,0,0,0,0,0,0,0,0],
                 [0,0,255,255,255,255,255,255,0],
                 [255,0,255,255,255,255,255,255,0],
                 [0,0,255,255,255,255,0,255,0],
                 [0,0,255,255,255,255,255,255,0],
                 [0,0,0,0,0,0,0,0,0],
                 [0,0,255,0,0,0,0,0,0]])
im_bw = test

In [5]:
def zero_pad (image, pad):
    paddedImage = np.pad(image,((pad,pad),(pad,pad)), 'constant', constant_values = (0,0))
    return paddedImage

In [6]:
def form_marker(image):
    H, W = np.shape(image)
    marker = np.empty([H, W])
    for h in range(H):                             
            for w in range(W):                  
                if h == 0 or h == (H-1) or w == 0 or w == (W-1):
                    marker[h,w] = 255 - image[h,w]
                else:
                    marker[h,w] = 0
    return marker

In [7]:
def dilation(image):
    H, W = np.shape(image)
    dilated_image = np.empty([H, W])
    paddedImage = zero_pad(image, 1)
    for h in range(H):                             
            for w in range(W):                  
                vert_start = h 
                vert_end = vert_start+3
                horiz_start = w
                horiz_end = horiz_start+3
                imageSlice = paddedImage[vert_start:vert_end, horiz_start:horiz_end]
                if np.sum(imageSlice) != 0:
                    dilated_image[h,w] = 255
                else:
                    dilated_image[h,w] = 0
    dilated_image = dilated_image.astype(np.uint8)
    return dilated_image

In [24]:
def erosion(image):
    Structuring_Element = np.full((7,7),255)
    H, W = np.shape(image)
    eroded_image = np.empty([H, W])
    for h in range(H):                             
            for w in range(W):                  
                vert_start = h-1 
                vert_end = vert_start+7
                horiz_start = w-1
                horiz_end = horiz_start+7
                imageSlice = image[vert_start:vert_end, horiz_start:horiz_end]
                if np.array_equal(imageSlice,Structuring_Element):
                    eroded_image[h,w] = 255
                else:
                    eroded_image[h,w] = 0
    eroded_image = eroded_image.astype(np.uint8)
    return eroded_image

In [9]:
def complement(marker):
    H, W = np.shape(marker)
    comp = np.empty([H, W])
    for h in range(H):                             
            for w in range(W):                  
                if marker[h,w] == 0:
                    comp[h,w] = 255
                else:
                    comp[h,w] = 0
    return comp

In [10]:
def intersection(image1, image2):
    H, W = np.shape(image1)
    intersect_array = np.empty([H, W])
    for h in range(H):                             
            for w in range(W):                  
                if image1[h,w] == image2[h,w] == 255:
                    intersect_array[h,w] = 1
                else:
                    intersect_array[h,w] = 0
    return intersect_array

In [11]:
def geodesicdilation(marker, mask, k):
    H, W = np.shape(marker)
    gdilation = np.empty([H, W])
    temp = marker
    for i in range(k):
        dilated_image = dilation(temp)
        intersect_array = intersection(dilated_image, mask)
        for h in range(H):                             
            for w in range(W):                  
                if intersect_array[h,w] == 1:
                    gdilation[h,w] = 255
                else:
                    gdilation[h,w] = 0
        temp = gdilation
    return gdilation, intersect_array        

In [25]:
#if the image object is black
#im_bw = form_mask(im_bw)
#im_bw = im_bw.astype(np.uint8)
eroded_image = erosion(im_bw)
boundary = im_bw-eroded_image
marker_image = form_marker(boundary)
marker_image = marker_image.astype(np.uint8)
mask = complement(boundary)
mask = mask.astype(np.uint8)
#print (im_bw)
#print (marker_image)

In [26]:
reconstruct, intersect_array = geodesicdilation(marker_image, mask, 1000)
intersect_array = intersect_array*255
intersect_array = intersect_array.astype(np.uint8)
reconstruct = reconstruct.astype(np.uint8)
output = complement(reconstruct)

In [133]:
#print(eroded_image)
#print(boundary)
print(reconstruct)

[[255 255 255 255 255 255   0 255 255]
 [255 255 255 255 255 255 255 255 255]
 [255 255   0   0   0   0   0   0 255]
 [  0 255   0   0   0   0   0   0 255]
 [255 255   0   0   0   0   0   0 255]
 [255 255   0   0   0   0   0   0 255]
 [255 255 255 255 255 255 255 255 255]
 [255 255   0 255 255 255 255 255 255]]


In [31]:
# im_bw = form_mask(im_bw)
# im_bw = im_bw.astype(np.uint8)
# new = im_bw
# for i in range(10):
#     dilated_image = dilation(new)
#     new = dilated_image   
#A = np.full((3,3),255)
#B = np.array([[255,255,255],[255,255,255],[255,255,255]])
#print(np.array_equal(A,B))

In [56]:

print(output.shape)
print(type(reconstruct[0,0]))
print(type(output[0,0]))

(1250, 1000)
<class 'numpy.uint8'>
<class 'numpy.uint8'>


In [28]:
output = output.astype(np.uint8)
cv2.imshow(' ',output)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [29]:
cv2.imwrite('fingerprint1000.png', output)

True